In [1]:
import pandas as pd
import re
import os
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [6]:
from selenium.webdriver import Chrome
#browser = Chrome() 

In [10]:
def preprocessing_articles(data):
    contents = []
    contents_new = []

    for content in tqdm(data['content']):

        content = content.split('\n')
        content = list(filter(lambda s: len(s) > 3, content))
        
        # 글 마지막에 항상 기자의 메일 주소가 추가되어 있다. 그 이후로 나오는 글은 불필요하므로 삭제함
        for i in range(1, len(content)):
            if '@' in content[-i]:
                break
        if i == len(content)-1:
            i = 1
        content_prep = content[:-i]

        # 너무 짧거나, 광고, copyright, 사진설명, 메일 등 불필요한 단어가 포함되어 있을 경우 불필요한 문장으로 판단하고 삭제 처리
        content_prep = list(filter(lambda s: len(s) > 30 and \
            '▶' not in s and \
                '©' not in s and \
                    '▲' not in s and \
                        '사진' not in s and \
                            '@' not in s and \
                                not re.findall("기자 *$", s) and \
                                    not re.findall("제공 *$", s) and \
                                        not re.findall("기자 *= *", s) , content_prep))
        
        # 언론사명이 들어간 prefix 삭제
        content_prep = list(map(lambda s: re.sub("^\[.*\]", "", s), content_prep)) #[스포츠서울]
        content_prep = list(map(lambda s: re.sub("^\(.*\)", "", s), content_prep)) #(서울=뉴시스1)
        
        contents.append('\n'.join(content))
        contents_new.append('\n'.join(content_prep))

    data['content'] = contents
    data['content_prep'] = contents_new

    return data

# train data

In [22]:
df = pd.read_csv('train_data.csv',index_col=0)

df['title'] = df['title'].apply(lambda x: x.replace("종합"," ").strip())

df.title[61]

'靑 6자회담 틀 완전히 무시하는게 아니다'

In [43]:
browser = Chrome() 
url = 'https://daum.net/'
browser.get(url)
content=[]
init =pd.DataFrame({'title':[],'content':[]})

for title in tqdm(df.title): # 확인을 위해 5개씩만 진행
    search = browser.find_element_by_css_selector("#q.tf_keyword")
    try:
        search.clear()
    except:
        pass
    search.send_keys(title)
    search.submit()
    try:
        browser.find_elements_by_css_selector('.tab.tab_news')[0].click()
        links = browser.find_elements_by_css_selector("a.f_nb")
        link_list = []
        for a in links:
            link_list.append(a.get_attribute('href'))
        daum_link = [ x for x in link_list if 'daum' in x ]
        daum_link = daum_link[0]
        article=[]

        webpage = requests.get(daum_link,headers={'User-Agent':'Mozilla/5.0'})
        soup = BeautifulSoup(webpage.content,'html.parser',from_encoding='utf-8')
        try:
            body = soup.select_one('section') 
        except:
            pass
        try:
            article.append(body.text)
        except:
            pass
        d = pd.DataFrame({'title':title,'content':article})
        init=pd.concat([d,init])
    except:
        pass
        # 열었다 닫았다 해야할 듯.

100%|█████████████████████████████████████████████████████████████████████████| 45654/45654 [13:59:08<00:00,  1.10s/it]


In [8]:
init

,title,content
0,시진핑 트럼프에 중미 무역협상 조속 타결 희망,\n\n\n\n 트럼프-시진핑 정상회담 (PG) [최자윤 제작]...
1,시진핑 트럼프에 중미 무역협상 조속 타결 희망,\n\n\n\n 트럼프-시진핑 정상회담 (PG) [최자윤 제작]...
2,시진핑 트럼프에 중미 무역협상 조속 타결 희망,\n\n\n\n미중 무역전쟁을 종식하기 위한 미중 무역협상이 진행 중인 가운데 시진...
3,시진핑 트럼프에 중미 무역협상 조속 타결 희망,\n[서울신문]\n\n\n\n 시진핑(習近平) 중국 국가 주석....
4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,\n\n\n\n (사진=연합뉴스)\n \n\n...
5,시진핑 트럼프에 중미 무역협상 조속 타결 희망,\n\n\n\n미중 무역전쟁을 종식하기 위한 미중 무역협상이 진행 중인 가운데 시진...
6,시진핑 트럼프에 중미 무역협상 조속 타결 희망,\n미국과 중국이 워싱턴에서 막바지 무역협상을 벌이고 있는 가운데 시진핑 중국 국가...
7,시진핑 트럼프에 중미 무역협상 조속 타결 희망,"\n-트럼프 29일 시진핑과 통화 “ 무역협상 잘 진행되고 있다”\n-시진핑, “양..."
8,시진핑 트럼프에 중미 무역협상 조속 타결 희망,"\n트럼프 “협상에 큰 진전”\n시진핑 “조속한 합의 기대”\n中, 지적재산권법원 ..."
9,시진핑 트럼프에 중미 무역협상 조속 타결 희망,\n\n\n\n 도널드 트럼프(오른쪽) 미국 대통령이 4일(현지...


In [10]:
preprocessing_articles(init).content_prep.apply(lambda x: x.replace('\n',' '))[10]

'            도널드 트럼프(오른쪽) 미국 대통령이 4일(현지시간) 백악관에서 류허 중국 부총리와 만난 뒤 미중 무역협상에 대한 기자들의 질문에 답변하고 있다. /워싱턴=UPI연합뉴스             [서울경제] 이르면 이달 말께 미중 무역협상이 타결될 가능성이 커졌다.  도널드 트럼프 미국 대통령이 4일(현지시간) “중국과의 무역협상이 4주 내에 마무리될 수 있다”고 밝혔다.  로이터통신에 따르면 트럼프 대통령은 이날 무역협상을 위해 방미 중인 류허 중국 부총리와 백악관에서 면담을 한 뒤 협상 전망을 묻는 기자들에게 “우리는 아마도 4주 안에 알 게 될 것이다. (전망이) 매우 좋다”며 이같이 말했다.  트럼프 대통령은 “미중 간 무역합의가 매우 근접하고 있으나 아직 합의가 이뤄진 건 아니다”라면서도 “협상이 타결되면 우리는 정상회담을 가질 것”이라고 설명했다. 정상회담 장소는 워싱턴인 것으로 알려졌다. 트럼프 대통령은 남은 협상 난제가 무엇이냐는 질문에는 관세와 지적재산권 탈취, 합의이행 문제를 꼽았다. 그는 “진전이 빠른 속도로 이뤄지고 있으며 갈 길이 남아있지만 그다지 먼 길이 남아있는 것은 아니다”라고 강조했다.  시진핑 중국 국가주석도 무역협상의 조속한 타결을 희망한다는 친서를 류허 부총리를 통해 트럼프 대통령에게 전달했다. 시 주석은 친서에서 “양국은 경제무역 협의문의 중요한 문제에 관해 실질적인 진전을 이뤘다”며 “조속히 중미 경제무역 협의문에 대한 본 담판이 타결되기를 바란다”고 강조했다. 트럼프 대통령과 만난 류 부총리도 “협상에서 엄청난 진전이 있었다”고 밝혔다. '

In [12]:
preprocessing_articles(init)

,title,content,content_prep
0,시진핑 트럼프에 중미 무역협상 조속 타결 희망,트럼프-시진핑 정상회담 (PG) [최자윤 제작] 일러스트\n ...,트럼프-시진핑 정상회담 (PG) [최자윤 제작] 일러스트\n ...
1,시진핑 트럼프에 중미 무역협상 조속 타결 희망,트럼프-시진핑 정상회담 (PG) [최자윤 제작] 일러스트\n ...,트럼프-시진핑 정상회담 (PG) [최자윤 제작] 일러스트\n ...
2,시진핑 트럼프에 중미 무역협상 조속 타결 희망,미중 무역전쟁을 종식하기 위한 미중 무역협상이 진행 중인 가운데 시진핑 중국 국가주...,미중 무역전쟁을 종식하기 위한 미중 무역협상이 진행 중인 가운데 시진핑 중국 국가주...
3,시진핑 트럼프에 중미 무역협상 조속 타결 희망,[서울신문]\n 시진핑(習近平) 중국 국가 주석.AP 연합뉴스\...,시진핑(習近平) 중국 국가 주석.AP 연합뉴스\n미중 무역협상...
4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,(사진=연합뉴스)\n \n ...,미중 고위급 무역협상이 워싱턴에서 이어지고 있는 가운데 시진핑 ...
5,시진핑 트럼프에 중미 무역협상 조속 타결 희망,미중 무역전쟁을 종식하기 위한 미중 무역협상이 진행 중인 가운데 시진핑 중국 국가주...,미중 무역전쟁을 종식하기 위한 미중 무역협상이 진행 중인 가운데 시진핑 중국 국가주...
6,시진핑 트럼프에 중미 무역협상 조속 타결 희망,미국과 중국이 워싱턴에서 막바지 무역협상을 벌이고 있는 가운데 시진핑 중국 국가주석...,미국과 중국이 워싱턴에서 막바지 무역협상을 벌이고 있는 가운데 시진핑 중국 국가주석...
7,시진핑 트럼프에 중미 무역협상 조속 타결 희망,"-트럼프 29일 시진핑과 통화 “ 무역협상 잘 진행되고 있다”\n-시진핑, “양국 ...","-트럼프 29일 시진핑과 통화 “ 무역협상 잘 진행되고 있다”\n-시진핑, “양국 ..."
8,시진핑 트럼프에 중미 무역협상 조속 타결 희망,"트럼프 “협상에 큰 진전”\n시진핑 “조속한 합의 기대”\n中, 지적재산권법원 설립...",도널드 트럼프 미국 대통령과 시진핑(習近平) 중국 국가주석이 29일(현지시간) 전화...
9,시진핑 트럼프에 중미 무역협상 조속 타결 희망,도널드 트럼프(오른쪽) 미국 대통령이 4일(현지시간) 백악관에...,도널드 트럼프(오른쪽) 미국 대통령이 4일(현지시간) 백악관에...


In [59]:
pp = preprocessing_articles(init).reset_index(drop=True)

pp['content_prep'] = pp.content_prep.apply(lambda x: x.replace('\n',' ').strip())

# 대괄호 속 내용과 일러스트 단어 삭제
pp['content_prep'] = pp['content_prep'].apply(lambda x: re.sub(r'\[[^][]*\]', "", x).replace('일러스트',''))

100%|█████████████████████████████████████████████████████████████████████████| 44318/44318 [00:03<00:00, 11191.95it/s]


In [61]:
pp['content_prep'].nunique()

40459

In [72]:
pp = pp[pp['content_prep'] != ''][['title','content_prep']] # 내용이 없는 행 제거

pp.to_csv('train_news.csv')

# test data

In [4]:
dft = pd.read_csv('test_data.csv')

dft['title'] = dft['title'].apply(lambda x: x.replace("종합"," ").strip())

In [8]:
browser = Chrome() 
url = 'https://daum.net/'
browser.get(url)
content=[]
init =pd.DataFrame({'title':[],'content':[]})
for title in tqdm(dft.title): # 확인을 위해 5개씩만 진행
    try:
        search = browser.find_element_by_css_selector("#q.tf_keyword")
        try:
            search.clear()
        except:
            pass
        search.send_keys(title)
        search.submit()
        browser.find_elements_by_css_selector('.tab.tab_news')[0].click()
        links = browser.find_elements_by_css_selector("a.f_nb")
        link_list = []
        for a in links:
            link_list.append(a.get_attribute('href'))
        daum_link = [ x for x in link_list if 'daum' in x ]
        article=[]
        for link in daum_link:
            webpage = requests.get(link,headers={'User-Agent':'Mozilla/5.0'})
            soup = BeautifulSoup(webpage.content,'html.parser',from_encoding='utf-8')
            try:
                body = soup.select_one('section') 
            except:
                pass
            try:
                article.append(body.text)
            except:
                pass
        d = pd.DataFrame({'title':title,'content':article})
        init=pd.concat([d,init])
            # 열었다 닫았다 해야할 듯.
    except:
        pass

100%|████████████████████████████████████████████████████████████████████████████| 9131/9131 [5:37:04<00:00,  2.21s/it]


In [11]:
dd = preprocessing_articles(init).reset_index(drop=True)
dd['content_prep'] = dd.content_prep.apply(lambda x: x.replace('\n',' ').strip())

# 대괄호 속 내용과 일러스트 단어 삭제
dd['content_prep'] = dd['content_prep'].apply(lambda x: re.sub(r'\[[^][]*\]', "", x).replace('일러스트',''))

100%|██████████████████████████████████████████████████████████████████████████| 42459/42459 [00:05<00:00, 7389.22it/s]


In [12]:
dd['title'].nunique()

8797

In [13]:
dd = dd[dd['content_prep'] != ''][['title','content_prep']] # 내용이 없는 행 제거

dd.to_csv('test_news.csv')